## Importações necessárias

In [1]:
from time import time
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt 

# Classificadores
from sklearn.svm import SVC # SVM
from sklearn.neighbors import KNeighborsClassifier # KNN
from sklearn.naive_bayes import MultinomialNB # Naive Bayes

# Seleção de modelo e divisão de dataset
from sklearn.model_selection import GridSearchCV, train_test_split

# Dataset
from sklearn.datasets import load_iris

# Metricas
from sklearn.metrics import f1_score, accuracy_score, recall_score, precision_score, confusion_matrix

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

## Classificadores

### Naive Bayes

Um classificador que usa o teorema de Bayes. Sua classificação é realizada prevendo probabilidades de associação para cada classe, sendo que a probabilidade de um determinado registro ou ponto irá pertencer à uma determinada classe. A classe com maior probabilidade vai ser a casse mais provável do registro.


O teorema de Bayes funciona com probabilidade condicional, que nada mais é do que dado um evento que vai acontecer dado que algo já aconteceu. 


$$ P(y|X) = \frac{P(X|y)P(y)}{P(X)} $$

Na equação acima y e X são eventos, onde tenta-se descobrir a probabilidade de y dado o evento X, X pode ser chamado de evidência. P(y) é o prior de y, ou seja, a probabilidade de y antes de ter uma evidência. P(y|X) é a probabilidade posterior de B, probabilidade após ter visto a evidência.


### K nearest neighborhood

O algoritmo KNN busca sempre classificar uma entrada a partir dos K vizinhos mais próximo, ou seja, se K = 15, então o modelo pegará o dado de entrada e fará um circulo até encontrar os 15 vizinhos mais próximos, a entrada então vai ser classificada pela maior quantidade de vizinhos de uma determinada classe ao seu redor, pois sua confiança vai ser gerada pelo cálculo:

$$ max(\frac{ClassePróxima}{K}) $$

<table><tr>
    <td> <img src="knn1.png" /> </td>
    <td>  <img src="knn2.png" />  </td>
</tr></table>

### Support Vector Machine

SVM é um algoritmo de aprendizagem de máquina supervisionado que pode ser utilizado tanto para regressão quanto classificação, sendo o último usado mais frequentemente. O algoritmos consiste em colocar os dados como sendo pontos em um plano cartesiano de n dimensões, onde cada característica é uma coordenada diferente no plano, após isso é performado uma busca do que é conhecido como hiperplano que diferencia as classes para performar a classificação.

<img src="svc.png" />

In [2]:
CLASSIFIERS = {
    "Naive Bayes": {
        "clf": MultinomialNB(),
        "params": {'alpha':[1, 10]}
    },
    
    "K-NN": {
        "clf": KNeighborsClassifier(),
        "params": {'n_neighbors':[5, 10, 15]}
    },
    
    "SVM": {
        "clf": SVC(),
        "params": {'kernel':('linear', 'rbf')}
    },
}

##  Manipulação do dataset

O conjunto de dados para exemplificação dos algoritmos anteriormentes citados foi retirado do kaggle e se trata de dados dos <a src= "https://www.kaggle.com/abcsds/pokemon/downloads/pokemon.zip/2">pokemons</a>. É possível encontrar o nome do pokémon, seus tipos, atributos e se ele é lendário ou não.

A proposta é conseguir por meio de outras caraterísticas ver se é possível dizer se o pokémon é ou não lendário utilizando só os tipos de classificadores citados nesse trabalho.

<img src="pokemons_lendários.jpeg">

In [3]:
df = pd.read_csv('Pokemon.csv')

df.head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


In [4]:
df.apply(lambda x: x.count())

#             800
Name          800
Type 1        800
Type 2        414
Total         800
HP            800
Attack        800
Defense       800
Sp. Atk       800
Sp. Def       800
Speed         800
Generation    800
Legendary     800
dtype: int64

In [5]:
df.drop(['#', 'Name', 'Type 2', 'Generation'], axis=1, inplace=True)

df.head()

,Type 1,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Legendary
0,Grass,318,45,49,49,65,65,45,False
1,Grass,405,60,62,63,80,80,60,False
2,Grass,525,80,82,83,100,100,80,False
3,Grass,625,80,100,123,122,120,80,False
4,Fire,309,39,52,43,60,50,65,False


In [6]:
df.describe()

,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed
count,800.00000,800.000000,800.000000,800.000000,800.000000,800.000000,800.000000
mean,435.10250,69.258750,79.001250,73.842500,72.820000,71.902500,68.277500
std,119.96304,25.534669,32.457366,31.183501,32.722294,27.828916,29.060474
min,180.00000,1.000000,5.000000,5.000000,10.000000,20.000000,5.000000
25%,330.00000,50.000000,55.000000,50.000000,49.750000,50.000000,45.000000
50%,450.00000,65.000000,75.000000,70.000000,65.000000,70.000000,65.000000
75%,515.00000,80.000000,100.000000,90.000000,95.000000,90.000000,90.000000
max,780.00000,255.000000,190.000000,230.000000,194.000000,230.000000,180.000000


In [7]:
pokemon_types = pd.get_dummies(df['Type 1'])
new_df = pd.concat([pokemon_types, df.drop('Type 1', axis=1)], axis=1)

new_df.head()

,Bug,Dark,Dragon,Electric,Fairy,Fighting,Fire,Flying,Ghost,Grass,...,Steel,Water,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Legendary
0,0,0,0,0,0,0,0,0,0,1,...,0,0,318,45,49,49,65,65,45,False
1,0,0,0,0,0,0,0,0,0,1,...,0,0,405,60,62,63,80,80,60,False
2,0,0,0,0,0,0,0,0,0,1,...,0,0,525,80,82,83,100,100,80,False
3,0,0,0,0,0,0,0,0,0,1,...,0,0,625,80,100,123,122,120,80,False
4,0,0,0,0,0,0,1,0,0,0,...,0,0,309,39,52,43,60,50,65,False


In [8]:
X_train, X_test, Y_train, Y_test = train_test_split(new_df.iloc[:,0:-1], df['Legendary'])

## Teste com diferentes algortimos

### Validação Cruzada

A validaçao cruzada é uma técnica de validação do modelo que busca ver como ele vai generalizar com um conjunto de dados. Essa técnica consiste em separar um conjunto de dados para teste durante o treinamento, esse conjunto é conhecido como conjunto de validação, essa separação é feita para limitar alguns problemas como o overfitting, underfitting e ainda dá uma visão de como o conjunto de dados vai se sair na generalização.

### Grid Search 

É um método para otimização de hiper parâmetro, ou seja, é encontrar a melhor combinação de hiper parâmetros para um determinado modelo e possibilitando a minimização do erro em um determinado conjunto de dados. Validação cruzada é utilizada para estimar a perfomance geral no conjunto de dados.

In [9]:
def best_estimator(descr, X_train, Y_train):
    
    gsclf = GridSearchCV(descr['clf'], descr['params'], cv=10)
    gsclf.fit(X_train, Y_train)
    
    return gsclf.best_estimator_

In [10]:
last_key = list(CLASSIFIERS.keys())[-1]

for name, descr in CLASSIFIERS.items():
    print(f"\nIniciando: {name}\n")
    
    start = time()
    
    clf = best_estimator(descr, X_train, Y_train)
    predicted = clf.predict(X_test)
    
    print(f"--> Melhores parâmetros ({name}):\n\n {clf}")
    
    print("\n--> Métricas\n")

    print(f"\tAcurácia: {accuracy_score(Y_test,predicted):.2f}")
    print(f"\tPrecisão: {precision_score(Y_test,predicted, average='macro'):.2f}")
    print(f"\tRevocação: {recall_score(Y_test,predicted, average='macro'):.2f}")
    print(f"\tEscore F1: {f1_score(Y_test,predicted, average='macro'):.2f}")
    
    print("\n--> Matrix de confusão\n")
    
    print(confusion_matrix(predicted, Y_test))
    
    end = time()
    print(f"\nfim em {end - start:.2f} segundos")
    
    if not name == last_key:
        print("\n######################################################################################")


Iniciando: Naive Bayes

--> Melhores parâmetros (Naive Bayes):

 MultinomialNB(alpha=10, class_prior=None, fit_prior=True)

--> Métricas

	Acurácia: 0.83
	Precisão: 0.59
	Revocação: 0.64
	Escore F1: 0.60

--> Matrix de confusão

[[160  10]
 [ 23   7]]

fim em 0.09 segundos

######################################################################################

Iniciando: K-NN

--> Melhores parâmetros (K-NN):

 KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=10, p=2,
           weights='uniform')

--> Métricas

	Acurácia: 0.96
	Precisão: 0.92
	Revocação: 0.85
	Escore F1: 0.88

--> Matrix de confusão

[[181   5]
 [  2  12]]

fim em 0.31 segundos

######################################################################################

Iniciando: SVM

--> Melhores parâmetros (SVM):

 SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  k